<a href="https://colab.research.google.com/github/rvidals/analise-exploratoria-cv-homogeneidade-triplice-viral-d1-d2/blob/main/AnaliseExploratoria_Homogeniedade_Imunobiologicos_TripliceViral_D1_D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importando as libs pandas e numpy
import pandas as pd
import numpy as np

In [ ]:
#Importando a lib Google Colab e o módulo files para "puxar" os arquivos
from google.colab import files
files.upload()

In [ ]:
#Salvando os arquivos em variáveis DF
tvd1 = pd.read_csv('/content/Tríplice_Viral_D1_cpnibr16654099173.csv', sep=";", encoding="latin");
tvd2 = pd.read_csv('/content/Tríplice_Viral_D2_cpnibr16654100014.csv', sep=";", encoding="latin");
microdado = pd.read_csv('/content/microdadoDropDuplicate.csv', encoding='latin');
print(f"tvd1: {tvd1.shape}\ntvd2:{tvd2.shape}\nmicrodados: {microdado.shape}");

tvd1: (5568, 4)
tvd2:(5553, 4)
microdados: (5570, 7)


In [ ]:
tvd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CD_MN      5568 non-null   int64 
 1   Município  5568 non-null   object
 2   2020       5568 non-null   object
 3    Total     5568 non-null   object
dtypes: int64(1), object(3)
memory usage: 174.1+ KB


In [ ]:
ano = tvd1.columns[2]
ano

'2020'

In [ ]:
tvd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5553 entries, 0 to 5552
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CD_MN      5553 non-null   int64 
 1   Município  5553 non-null   object
 2   2020       5553 non-null   object
 3    Total     5553 non-null   object
dtypes: int64(1), object(3)
memory usage: 173.7+ KB


In [ ]:
#Para fazer o marge as colunas precisam ter o mesmo tipo de dado 
#Transformando de string para inteiro e pegando somente os 6 primeiros números
#Padrão do código municipal do SUS
microdado['CD_MN'] = [int(str(x)[0:6]) for x in  microdado['CO_MUNICIPIO']]

In [ ]:
#A chave para o merge entre as tabelas é a columna CD_MN
tvd1_ =  pd.merge(
    microdado,tvd1,
    how='left', 
    on=["CD_MN"]
);

tvd2_ =  pd.merge(
    microdado,tvd2, 
    how='left',
    on=["CD_MN"]
);

TVD2 = tvd2_.rename(columns={'2020':'TVD2'});
TVD1 = tvd1_.rename(columns={'2020':'TVD1'});

In [ ]:
TVD2

,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,CD_MN,Município,TVD2,Total,TVD2_B
0,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,110001,110001 ALTA FLORESTA D\'OESTE,79.38,"79,38",0
1,Norte,1,Rondônia,RO,11,Ariquemes,1100023,110002,110002 ARIQUEMES,73.43,"73,43",0
2,Norte,1,Rondônia,RO,11,Cabixi,1100031,110003,110003 CABIXI,95.52,"95,52",1
3,Norte,1,Rondônia,RO,11,Cacoal,1100049,110004,110004 CACOAL,56.89,"56,89",0
4,Norte,1,Rondônia,RO,11,Cerejeiras,1100056,110005,110005 CEREJEIRAS,84.84,"84,84",0
...,...,...,...,...,...,...,...,...,...,...,...,...
5565,Centro-Oeste,5,Goiás,GO,52,Vianópolis,5222005,522200,522200 VIANOPOLIS,87.76,"87,76",0
5566,Centro-Oeste,5,Goiás,GO,52,Vicentinópolis,5222054,522205,522205 VICENTINOPOLIS,71.82,"71,82",0
5567,Centro-Oeste,5,Goiás,GO,52,Vila Boa,5222203,522220,522220 VILA BOA,72.13,"72,13",0
5568,Centro-Oeste,5,Goiás,GO,52,Vila Propício,5222302,522230,522230 VILA PROPICIO,80.30,"80,30",0


ÁREA RESERVADA PARA VISUALIZAR AS INFORMAÇÕES PRESENTES NO DATA FRAME

*   AS COLUNAS QUE FORAM CRIADAS E PREENCHIDAS USANDO O 
> List comprehension in python with if else: [output if condition else output for l in list]



In [ ]:
#Comprehension para subistituir "," para "."
#Importante: o else é importante porque há valores que já estão no formato desejavel
TVD1['TVD1'] = [x.replace(',','.') if type(x) == str else x for x in TVD1['TVD1']]
TVD2['TVD2'] = [x.replace(',','.') if type(x) == str else x for x in TVD2['TVD2']]

In [ ]:
#Após a substituição é possível transformar  os valores para float
TVD2['TVD2'] = TVD2['TVD2'].astype(float)
TVD1['TVD1'] = TVD1['TVD1'].astype(float)

In [ ]:
#Criação de uma coluna com valores de 0 e 1, a partir da classificação 
#Greater than (maior que) or igual (igual) a 95.00
#Meta para a cobertura de Tríplice Viral D1 e D2
TVD1['TVD1_B'] = [ 1 if x >= 95.00 else 0 for x in TVD1['TVD1']];
TVD2['TVD2_B'] = [ 1 if x >= 95.00 else 0 for x in TVD2['TVD2']];


In [ ]:
TVD1['TVD1_B'].value_counts()

0    3151
1    2419
Name: TVD1_B, dtype: int64

In [ ]:
TVD2['TVD2_B'].value_counts()

0    4258
1    1312
Name: TVD2_B, dtype: int64

In [ ]:
a = TVD1['TVD1'] >=95.00
b = TVD2['TVD2'] >=95.00


In [ ]:
a.value_counts()

False    3151
True     2419
Name: TVD1, dtype: int64

In [ ]:
b.value_counts()

False    4258
True     1312
Name: TVD2, dtype: int64

In [ ]:
TVD2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NO_REGIAO     5570 non-null   object 
 1   CO_REGIAO     5570 non-null   int64  
 2   NO_UF         5570 non-null   object 
 3   SG_UF         5570 non-null   object 
 4   CO_UF         5570 non-null   int64  
 5   NO_MUNICIPIO  5570 non-null   object 
 6   CO_MUNICIPIO  5570 non-null   int64  
 7   CD_MN         5570 non-null   int64  
 8   Município     5375 non-null   object 
 9   TVD2          5375 non-null   float64
 10  TVD2_B        5570 non-null   int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 522.2+ KB


# Nesta parte foi utilizada a função pivot_table() do Pandas, em que pode ser feito várias operações de agregação em colunas diferentes

In [ ]:
#Salvando o resultado na variável TVD1_UF
TVD1_UF = pd.pivot_table(
   TVD1,
   index=['NO_REGIAO','CO_UF','SG_UF','NO_UF'],
   aggfunc={'TVD1_B': np.sum, 'CD_MN': np.count_nonzero}
).rename(columns={'CD_MN': 'QTD_MN', 'TVD1_B':'ge95'}).sort_values(by='CO_UF').reset_index()


In [ ]:
TVD1_UF

,NO_REGIAO,CO_UF,SG_UF,NO_UF,QTD_MN,ge95
0,Norte,11,RO,Rondônia,52,20
1,Norte,12,AC,Acre,22,2
2,Norte,13,AM,Amazonas,62,10
3,Norte,14,RR,Roraima,15,1
4,Norte,15,PA,Pará,144,11
5,Norte,16,AP,Amapá,16,3
6,Norte,17,TO,Tocantins,139,59
7,Nordeste,21,MA,Maranhão,217,45
8,Nordeste,22,PI,Piauí,224,98
9,Nordeste,23,CE,Ceará,184,68


In [ ]:
#Salvando o resultado na variável TVD1_UF
TVD2_UF = pd.pivot_table(
   TVD2,
   index=['NO_REGIAO','CO_UF','SG_UF','NO_UF'],
   aggfunc={'TVD2_B': np.sum, 'CD_MN': np.count_nonzero}
).rename(columns={'CD_MN': 'QTD_MN', 'TVD2_B':'ge95'}).sort_values(by='CO_UF').reset_index()


In [ ]:
TVD2_UF

,NO_REGIAO,CO_UF,SG_UF,NO_UF,QTD_MN,ge95
0,Norte,11,RO,Rondônia,52,11
1,Norte,12,AC,Acre,22,0
2,Norte,13,AM,Amazonas,62,1
3,Norte,14,RR,Roraima,15,1
4,Norte,15,PA,Pará,144,6
5,Norte,16,AP,Amapá,16,1
6,Norte,17,TO,Tocantins,139,35
7,Nordeste,21,MA,Maranhão,217,17
8,Nordeste,22,PI,Piauí,224,34
9,Nordeste,23,CE,Ceará,184,21


In [ ]:
#Contando os valores da coluna ge95 > maior e igual a 95%
TVD1_UF['ge95'].value_counts() 

1     8
0     5
5     2
4     2
6     2
2     1
43    1
18    1
13    1
25    1
55    1
8     1
15    1
Name: ge95, dtype: int64

In [ ]:
#Somando os valores da coluna ge95 > maior e igual a 95%
TVD1_UF['ge95'].sum()

217

In [ ]:
#Contando os valores da coluna ge95 > maior e igual a 95%
TVD2_UF['ge95'].value_counts()

0    16
1     7
5     1
4     1
8     1
6     1
Name: ge95, dtype: int64

In [ ]:
#Somandp os valores da coluna ge95 > maior e igual a 95%
TVD2_UF['ge95'].sum()

30

In [ ]:
#Informações sobre o dataframe
TVD2_UF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NO_REGIAO      27 non-null     object 
 1   CO_UF          27 non-null     int64  
 2   SG_UF          27 non-null     object 
 3   NO_UF          27 non-null     object 
 4   QTD_MN         27 non-null     int64  
 5   ge95           27 non-null     int64  
 6   Homegeneidade  27 non-null     float64
dtypes: float64(1), int64(3), object(3)
memory usage: 1.6+ KB


In [ ]:
#Criando a coluna de Homogeneidade e populando com o calculo no comprehension
TVD1_UF['Homegeneidade'] = [round((p['ge95'] * 100) / p['QTD_MN'] ,2) for x, p in TVD1_UF.iterrows()]
TVD2_UF['Homegeneidade'] = [round((p['ge95'] * 100) / p['QTD_MN'] ,2) for x, p in TVD2_UF.iterrows()]

In [ ]:
#Dataframe final
TVD1_UF

,NO_REGIAO,CO_UF,SG_UF,NO_UF,QTD_MN,ge95,Homegeneidade
0,Norte,11,RO,Rondônia,52,20,38.46
1,Norte,12,AC,Acre,22,2,9.09
2,Norte,13,AM,Amazonas,62,10,16.13
3,Norte,14,RR,Roraima,15,1,6.67
4,Norte,15,PA,Pará,144,11,7.64
5,Norte,16,AP,Amapá,16,3,18.75
6,Norte,17,TO,Tocantins,139,59,42.45
7,Nordeste,21,MA,Maranhão,217,45,20.74
8,Nordeste,22,PI,Piauí,224,98,43.75
9,Nordeste,23,CE,Ceará,184,68,36.96


In [ ]:
#Função de homogeneidade geral - Brasil
def homogeneidade_br(col_nmMunicipio: float, col_ge95: float):
  nm_municipio = col_nmMunicipio.sum()
  nm_ge95 = col_ge95.sum()
  resultado = (nm_ge95  / nm_municipio ) * 100
  return f"Homogeneidade para o Brasil: {resultado:.2f}"

'Homogeneidade para o Brasil: 23.55'

In [ ]:
homogeneidade_br(TVD1_UF['QTD_MN'],TVD1_UF['ge95'])

'Homogeneidade para o Brasil: 43.43'

In [ ]:
homogeneidade_br(TVD2_UF['QTD_MN'],TVD2_UF['ge95'])

'Homogeneidade para o Brasil: 23.55'

In [ ]:
#Calculo de validação para a homogeneidade do Brasil
a = TVD1_UF['QTD_MN'].sum();
b = TVD1_UF['ge95'].sum();
c = (b * 100) / a
print(round(c,2))

43.43


In [ ]:
#Calculo de validação para a homogeneidade do Brasil
a = TVD2_UF['QTD_MN'].sum();
b = TVD2_UF['ge95'].sum();
c = (b * 100) / a
print(round(c,2))

23.55


In [ ]:
#Salvando os resultado em csv
TVD1_UF.to_csv('Homogeneidade de TVD1 por UF.csv', index=False, encoding='latin');
TVD2_UF.to_csv('Homogeneidade de TVD2 por UF.csv', index=False, encoding='latin');
